In [1]:
import numpy as np
from numpy import nan
import pandas as pd

In [2]:
key = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'conversions')
df = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'raw data')

def create_dict_key(dataframe):
    '''
    '''
    conversion_key = dict()
    for year in dataframe.columns[1:]:
        values = list(dataframe[year])
        categories = list(dataframe['categories'])
        temp = dict()
        for tup in tuple(zip(categories, values)):
            temp[tup[0]] = tup[1]
        
        temp['USD'] = 1
        conversion_key[year] = temp
            
    return conversion_key

conversion_key = create_dict_key(key)


def adjustments(raw_data, conversion_key, fy_starts=13):
    '''
    Takes inputs of raw_data and conversion key and
    converts to real money terms.
    
    Inputs:
        raw_data: pandas dataframe of raw data Excel
        raw_data: pandas dataframe of raw data Excel
        
    Outputs:
        adjusted_data: pandas dataframe of adjusted data
    '''
    columns = list(raw_data.columns[fy_starts:])
    raw_data['Region'] = raw_data['Type']
        
    for index, row in raw_data.iterrows():
        currency = row['Currency']
        for year in columns:
            year_match = int(year[2:])
            infl = conversion_key[year_match]['Inflation Adjustment']
            curr = conversion_key[year_match][currency]
            if currency == 'Yen':
                row[year] = (float(row[year])/(1000/curr))*infl
            else:
                row[year] = (float(row[year])/curr)*infl
            
        raw_data.loc[index] = row
        
    clean = raw_data
    clean['Source'] = clean['Source'].str.replace(" ", "")
    
    return clean[['Proper Name', 'Source'] + columns]

clean = adjustments(df, conversion_key)

In [15]:
clean

,Proper Name,Source,FY2007,FY2008,FY2009,FY2010,FY2011,FY2012,FY2013,FY2014,...,FY2029,FY2030,FY2031,FY2032,FY2033,FY2034,FY2035,FY2036,FY2037,FY2038
0,Actemra,Actemra1,0,0.0,0.0,519.382885,913.806742,1158.377021,1424.369677,1677.279130,...,1026.982709,950.614498,983.526357,1026.718156,1067.725867,1061.260590,1244.462648,NaN,NaN,NaN
1,Actemra,Actemra2,0,0.0,0.0,75.879615,208.489888,331.554468,431.294194,556.352391,...,335.725889,315.654785,335.012030,343.125495,362.802547,358.359781,363.288261,NaN,NaN,NaN
2,Adempas,Adempas1,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,84.362632,...,238.420843,234.891855,204.027394,86.399891,71.240784,59.505770,26.347257,NaN,NaN,NaN
3,Adempas,Adempas2,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,186.799505,148.147578,95.047725,96.087403,95.873187,30.790721,21.553504,NaN,NaN,NaN
4,Adempas,Adempas3,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,202.025598,186.791391,205.389501,200.182090,200.134538,57.377336,40.164135,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,Xtandi,Xtandi4,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
214,Xtandi,Xtandi5,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
215,Xtandi,Xtandi6,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,982.085300,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
216,Yervoy,Yervoy1,0,0.0,0.0,0.000000,473.760000,912.999200,1226.304000,1648.995600,...,1052.208894,859.211704,529.341355,451.136010,362.668629,615.523927,600.701670,NaN,NaN,NaN


In [3]:
key = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'dictionary')

def split_strings(string):
    '''
    '''
    new_string = '{'
    if string == '':
        return string
    for char in string:
        if char not in ['+', '-', '*', '/', '(', ')']:
            new_string += char
        else:
            new_string += '}'
            new_string += char
            new_string += '{'
    
    new_string += '}'
    new_string = new_string.replace("{}", "" )
    
    return new_string.replace("{0.5}", "0.5" )


def formulas(dataframe):
    '''
    '''
    dataframe["formula"] = dataframe["formula"].apply(split_strings)
    formula_key = dict()
    for _, row in dataframe.iterrows():
        formula = row['formula']
        formula_key[row['ID']] = formula

    return formula_key

formula_key = formulas(key)

In [28]:
formula_key

{'Actemra WW': '{Actemra1}',
 'Actemra US': '{Actemra2}',
 'Actemra Ex-US': '{Actemra1}-{Actemra2}',
 'Adempas WW': '{Adempas1}+{Adempas2}',
 'Adempas US': '{Adempas1}',
 'Adempas Ex-US': '{Adempas2}',
 'Aimovig WW': '{Aimovig1}',
 'Aimovig US': '{Aimovig2}',
 'Aimovig Ex-US': '{Aimovig1}-{Aimovig2}',
 'Aristada WW': '{Aristada1}',
 'Aristada US': '{Aristada1}',
 'Aristada Ex-US': '{Nan}',
 'Aubagio WW': '{Aubagio1}',
 'Aubagio US': '{Aubagio2}',
 'Aubagio Ex-US': '{Aubagio1}-{Aubagio2}',
 'Austedo WW': '{Austedo1}',
 'Austedo US': '{Austedo2}',
 'Austedo Ex-US': '{Austedo1}-{Austedo2}',
 'Biktarvy WW': '{Biktarvy1}',
 'Biktarvy US': '{Biktarvy2}',
 'Biktarvy Ex-US': '{Biktarvy1}-{Biktarvy2}',
 'Breo Ellipta WW': '{BreoEllipta1}',
 'Breo Ellipta US': '{BreoEllipta2}',
 'Breo Ellipta Ex-US': '{BreoEllipta1}-{BreoEllipta2}',
 'Brilinta WW': '{Brilinta1}',
 'Brilinta US': '{Brilinta2}',
 'Brilinta Ex-US': '{Brilinta1}-{Brilinta2}',
 'Calquence WW': '{Calquence1}',
 'Calquence US': '{Calqu

In [4]:
def evaluate_helper(clean_dataframe):
    '''
    '''
    holder = dict()
    for index, row in clean_dataframe.iterrows():
        for year in clean_dataframe.columns[2:]:
            if year not in holder:
                holder[year] = {row['Source'] : row[year]}
            else:
                temp = holder.get(year)
                holder[year].update({row['Source'] : row[year]})
                
    return holder

    
eval_map = evaluate_helper(clean)

def evaluate(eval_map, formula_key):
    '''
    '''
    return_dict = dict()
    for year in eval_map.keys():
        temp = eval_map.get(year)
        temp_dict = dict()
        for ID, formula in formula_key.items():
            if formula != '{Nan}':
                formula = formula.format(**temp)
                if formula != np.nan:
                    try:
                        result = eval(formula)
                    except ZeroDivisionError:
                        result = 0
                else:
                    result = formula
                temp_dict[ID] = result
            else:
                temp_dict[ID] = 'Nan'
                
        return_dict[year] = temp_dict
        
    return return_dict

evaluated_key = evaluate(eval_map, formula_key)


def turn_into_dataframe(evaluated_key):
    '''
    '''
    counter = 0 
    for year, values in evaluated_key.items():
        data = []
        for key, amount in values.items():
            data.append([key, amount])
        df_temp = pd.DataFrame(data, columns = ['Product Name', year])
        if counter > 0:
            df = df.merge(df_temp, how='inner', on='Product Name')
        else: 
            df = df_temp

        counter += 1
        
    return df
    
    
evaluated_key = turn_into_dataframe(evaluated_key)

In [5]:
def check_string(x, string_list):
    for string in string_list:
        if string in x:
            return string
    return None



def merge_final_clean(raw_dataframe, evaluated_key):
    '''
    '''
    cols_y = ['FY2007', 'FY2008', 'FY2009', 'FY2010', 'FY2011',
       'FY2012', 'FY2013', 'FY2014', 'FY2015', 'FY2016', 'FY2017', 'FY2018',
       'FY2019', 'FY2020', 'FY2021', 'FY2022', 'FY2023', 'FY2024', 'FY2025',
       'FY2026', 'FY2027', 'FY2028', 'FY2029', 'FY2030', 'FY2031', 'FY2032',
       'FY2033', 'FY2034', 'FY2035', 'FY2036', 'FY2037', 'FY2038']
    
    cols = ['Proper Name', 'Generic Name', 
            'Medicare Spend', 'Original Manufacturer', 
            'Application', 'Approval Date', 'Year']
    
    pnames = list(raw_dataframe['Proper Name'])
    temp = raw_dataframe[cols].drop_duplicates()
    
    evaluated_key["Proper Name"] = evaluated_key["Product Name"].apply(lambda x: check_string(x, pnames))
    
    final = temp.merge(evaluated_key, how='left', on='Proper Name')

    for col in cols_y:
        final = final.rename(columns={col: col[2:]})
    
    return final

    
final = merge_final_clean(df, evaluated_key)

In [24]:
final

,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Application,Approval Date,Year,Product Name,2007,2008,...,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038
0,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra WW,0,0.0,...,1026.982709,950.614498,983.526357,1026.718156,1067.725867,1061.26059,1244.462648,NaN,NaN,NaN
1,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra US,0,0.0,...,335.725889,315.654785,335.01203,343.125495,362.802547,358.359781,363.288261,NaN,NaN,NaN
2,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra Ex-US,0,0.0,...,691.25682,634.959712,648.514326,683.592661,704.92332,702.900809,881.174387,NaN,NaN,NaN
3,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas WW,0,0.0,...,425.220348,383.039432,299.075119,182.487295,167.113971,90.296491,47.900762,NaN,NaN,NaN
4,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas US,0,0.0,...,238.420843,234.891855,204.027394,86.399891,71.240784,59.50577,26.347257,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi US,0,0.0,...,13.176228,5.622733,2.839458,1.589025,1.244972,0.882307,NaN,NaN,NaN,NaN
278,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi Ex-US,0,0.0,...,23.470813,19.440799,15.082767,12.319778,5.1098,4.200429,NaN,NaN,NaN,NaN
279,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy WW,0,0.0,...,1052.208894,859.211704,529.341355,451.13601,362.668629,615.523927,600.70167,NaN,NaN,NaN
280,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy US,0,0.0,...,608.303033,475.140206,287.319594,242.144549,159.085817,182.393814,145.915051,NaN,NaN,NaN


In [6]:
def reformat_final(dataframe):
    '''
    '''
    data = []
    years = dataframe.columns[8:]
    other_info = dataframe.columns[:8]
    
    for index, row in dataframe.iterrows():
        new_data = list(row[other_info].values)
        for year in years:
            #if row[str(year)] != 0:
                #new_data.append(row[str(year)])
            if int(year) >= row['Year']:
                new_data.append(row[str(year)])
                
        data.append(new_data)
        
    longest_list = max(data, key=lambda x: len(x))
    num_years = len(longest_list) - 8
    year_cols = [f't{i}' for i in range(1, num_years+1)]
    cols = list(other_info) + year_cols
        
    data = pd.DataFrame(data, columns = cols)
    data[year_cols] = data[year_cols].astype(float)
    data['t31'] = np.nan

    return data.fillna(np.nan)


re_final = reformat_final(final)
re_final

,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Application,Approval Date,Year,Product Name,t1,t2,...,t22,t23,t24,t25,t26,t27,t28,t29,t30,t31
0,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra WW,519.382885,913.806742,...,983.526357,1026.718156,1067.725867,1061.260590,1244.462648,NaN,NaN,NaN,NaN,NaN
1,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra US,75.879615,208.489888,...,335.012030,343.125495,362.802547,358.359781,363.288261,NaN,NaN,NaN,NaN,NaN
2,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra Ex-US,443.503269,705.316854,...,648.514326,683.592661,704.923320,702.900809,881.174387,NaN,NaN,NaN,NaN,NaN
3,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas WW,0.000000,84.362632,...,90.296491,47.900762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas US,0.000000,84.362632,...,59.505770,26.347257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi US,0.000000,1.521026,...,1.244972,0.882307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi Ex-US,0.000000,0.000000,...,5.109800,4.200429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy WW,473.760000,912.999200,...,451.136010,362.668629,615.523927,600.701670,NaN,NaN,NaN,NaN,NaN,NaN
280,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy US,425.068000,650.479600,...,242.144549,159.085817,182.393814,145.915051,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def find_last_valid(row):
    last_valid = row.last_valid_index()
    return (last_valid, row[last_valid])

def pro_rate(dataframe):
    '''
    '''
    last_values = dataframe.apply(find_last_valid, axis=1).tolist()
    
    end_of_year = pd.to_datetime(dataframe['Approval Date'].dt.year.astype(str) + '-12-31')
    days_remaining = (end_of_year - dataframe['Approval Date']).dt.days
    dataframe['percent_remaining'] = (days_remaining / 365)
    dataframe['percent_missing'] = 1 - dataframe['percent_remaining']
    
    for year in range(1,31):
        var = 't' + str(year)
        var1 = 't' + str(year+1)
        dataframe[var] = (dataframe[var]) + (dataframe[var1]*dataframe['percent_missing'])
        dataframe[var1] -= (dataframe[var1]*dataframe['percent_missing'])
    
    for i, (col, val) in enumerate(last_values):
        dataframe.loc[dataframe.index[i], col] = val
    
    return dataframe


pro_rate = pro_rate(re_final)
pro_rate

,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Application,Approval Date,Year,Product Name,t1,t2,...,t24,t25,t26,t27,t28,t29,t30,t31,percent_remaining,percent_missing
0,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra WW,539.411526,919.167186,...,1067.584162,1065.275977,1244.462648,NaN,NaN,NaN,NaN,NaN,0.978082,0.021918
1,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra US,80.449257,211.187194,...,362.705171,358.467802,363.288261,NaN,NaN,NaN,NaN,NaN,0.978082,0.021918
2,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra Ex-US,458.962269,707.979993,...,704.878991,706.808175,881.174387,NaN,NaN,NaN,NaN,NaN,0.978082,0.021918
3,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas WW,64.947670,206.297003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.230137,0.769863
4,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas US,64.947670,177.325518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.230137,0.769863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi US,1.012628,4.439987,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.334247,0.665753
278,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi Ex-US,0.000000,0.914117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.334247,0.665753
279,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy WW,683.874884,985.102222,...,612.112777,600.701670,NaN,NaN,NaN,NaN,NaN,NaN,0.769863,0.230137
280,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy US,574.767415,668.967685,...,173.998701,145.915051,NaN,NaN,NaN,NaN,NaN,NaN,0.769863,0.230137


In [8]:
def apply_discount(dataframe, discount_rate=0.1):
    '''
    '''
    for year in range(1,31):
        var = 't' + str(year)
        dataframe[var] = dataframe[var]/(1+discount_rate)**(year-0.5)
        
    return dataframe

apply_discount(pro_rate)

,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Application,Approval Date,Year,Product Name,t1,t2,...,t24,t25,t26,t27,t28,t29,t30,t31,percent_remaining,percent_missing
0,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra WW,514.308710,796.719568,...,113.677371,103.119630,109.513685,NaN,NaN,NaN,NaN,NaN,0.978082,0.021918
1,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra US,76.705357,183.053717,...,38.621190,34.699991,31.969651,NaN,NaN,NaN,NaN,NaN,0.978082,0.021918
2,Actemra,Tocilizumab,2.821445e+08,Genentech,BLA,2010-01-08,2010,Actemra Ex-US,437.603353,613.665852,...,75.056181,68.419639,77.544035,NaN,NaN,NaN,NaN,NaN,0.978082,0.021918
3,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas WW,61.925173,178.814977,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.230137,0.769863
4,Adempas,Riociguat,4.631524e+08,Bayer,NDA,2013-10-08,2013,Adempas US,61.925173,153.702952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.230137,0.769863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi US,0.965503,3.848511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.334247,0.665753
278,Xtandi,Enzalutamide,1.968568e+09,Medivation,NDA,2012-08-31,2012,Xtandi Ex-US,0.000000,0.792342,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.334247,0.665753
279,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy WW,652.049118,853.871014,...,65.178347,58.148438,NaN,NaN,NaN,NaN,NaN,NaN,0.769863,0.230137
280,Yervoy,Ipilimumab,3.659614e+08,BristolMyers Squibb,BLA,2011-03-25,2011,Yervoy US,548.019228,579.850601,...,18.527546,14.124702,NaN,NaN,NaN,NaN,NaN,NaN,0.769863,0.230137
